# 1. Debugging

In [1]:
def increment_dictionary_values(d, i):
    """
    Increment by i each entry of the dictionary d.
    Note: working w/ a copy and not reference;no inplace operation
    """
    res = {}
    for k, v in d.items():
        res[k] = v + i 
    return res

In [2]:
import unittest

class TestIncrementDictionaryValues(unittest.TestCase):
    def test_increment_dictionary_values(self):
        d = {'a': 1}
        dd = increment_dictionary_values(d, 1)
        ddd = increment_dictionary_values(d, -1)
        self.assertEqual(dd['a'], 2)
        self.assertEqual(ddd['a'], 0)

suite = unittest.TestLoader().loadTestsFromTestCase(TestIncrementDictionaryValues)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

test_increment_dictionary_values (__main__.TestIncrementDictionaryValues) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.010s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

# 2. Write a function & tests

In [3]:
def calc_largest_loss(prices):
    """
    Returns the largest possible loss a client could have
    made with only a buy transaction followed by a sell
    transaction.
    """
    if not isinstance(prices, list):
        raise TypeError("Only lists accepted")
    if len(prices) < 2:
        raise ValueError("At least 2 prices are needed")

    min_price, max_price = min(prices), max(prices)

    if min_price < 0:
        raise ValueError("Only positive numbers accepted")
    
    return max_price - min_price

In [4]:
import unittest

class TestCalcLargestLoss(unittest.TestCase):
    def test_duplicates(self):
        prices = [10, 15] * 2
        largest_loss = calc_largest_loss(prices)
        self.assertEqual(largest_loss, 5)

    def test_one_buy_no_sell(self):
        with self.assertRaises(ValueError):
            calc_largest_loss([10.5])

    def test_low_prices(self):
        prices = [0.01, 0.02]
        largest_loss = calc_largest_loss(prices)
        self.assertEqual(largest_loss, 0.01)

    def test_not_list(self):
        with self.assertRaises(TypeError):
            calc_largest_loss({})

    def test_not_number(self):
        with self.assertRaises(TypeError):
            calc_largest_loss([0.01, "0.02"])

    def test_negative_price(self):
        prices = [0.01, -0.02]
        with self.assertRaises(ValueError):
            calc_largest_loss(prices)

    def test_no_prices(self):
        with self.assertRaises(ValueError):
            calc_largest_loss([])

    def test_no_diff(self):
        prices = [0.01, 0.01]
        largest_loss = calc_largest_loss(prices)
        self.assertEqual(largest_loss, 0)

    def test_very_large_loss(self):
        prices = [0.01, 1e12 + 0.02, 0.02, 1e11 + 0.02]
        largest_loss = calc_largest_loss(prices)
        self.assertEqual(largest_loss, 1e12 + 0.01)

suite = unittest.TestLoader().loadTestsFromTestCase(TestCalcLargestLoss)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

test_duplicates (__main__.TestCalcLargestLoss) ... ok
test_low_prices (__main__.TestCalcLargestLoss) ... ok
test_negative_price (__main__.TestCalcLargestLoss) ... ok
test_no_diff (__main__.TestCalcLargestLoss) ... ok
test_no_prices (__main__.TestCalcLargestLoss) ... ok
test_not_list (__main__.TestCalcLargestLoss) ... ok
test_not_number (__main__.TestCalcLargestLoss) ... ok
test_one_buy_no_sell (__main__.TestCalcLargestLoss) ... ok
test_very_large_loss (__main__.TestCalcLargestLoss) ... ok

----------------------------------------------------------------------
Ran 9 tests in 0.026s

OK


<unittest.runner.TextTestResult run=9 errors=0 failures=0>

# 3. Design a set of classes

In [5]:
import datetime

class SQL:
    """
    Class used as a Descriptor to get the raw SQL
    cf. https://docs.python.org/3/howto/descriptor.html
    """
    def __get__(self, obj, objtype=None):
        clauses = self._format_filters(obj.filters, obj._operators)
        return f"""SELECT * FROM MyTable{clauses};"""

    @staticmethod
    def _format_value(value):
        if isinstance(value, (str, datetime.date)):
            value = f"\'{value}\'"
        return value

    def _format_filters(self, filters, operators):
        if not len(filters):
            return ""

        # map operator name to the method
        mapping = {o: getattr(SQL, f"_{o}") for o in operators}
        clauses = [mapping[operator](field, value) for field, operator, value in filters]
        return " WHERE " + " AND ".join(clauses)

    @staticmethod
    def _in(field, value):
        return f"{field} IN {tuple(value)}"

    @staticmethod
    def _notin(field, value):
        return f"{field} NOT IN {tuple(value)}"

    @staticmethod
    def _eq(field, value):
        return f"{field} = {SQL._format_value(value)}"

    @staticmethod
    def _lt(field, value):
        return f"{field} < {SQL._format_value(value)}"

    @staticmethod
    def _gt(field, value):
        return f"{field} > {SQL._format_value(value)}"

class Query:
    _operators = ["in", "notin", "eq", "lt", "gt"]
    _fields = ["id", "url", "date", "rating"]
    # Descriptor
    sql = SQL()

    def __init__(self):
        self.filters = []

    def __str__(self):
        return self.sql

    def filter(self, **criterion):
        for k, v in criterion.items():
            field, operator = k.split("__")
            if operator not in self._operators:
                raise ValueError(f"Available operators: {self._operators}")
            if field not in self._fields:
                raise ValueError(f"Available fields: {self._fields}")
            if operator.endswith("in") and not isinstance(v, (list, tuple, set)):
                raise ValueError("Value must be a list, a tuple or a set")
            self.filters.append((field, operator, v))    
        return self

class MyTable(Query):
    id = 0
    url = ""
    date = ""
    rating = 0


In [6]:
import unittest


class TestQuery(unittest.TestCase):
    def test_wrong_field(self):
        with self.assertRaises(ValueError):
            MyTable().filter(job__in=[1])

    def test_wrong_method(self):
        with self.assertRaises(ValueError):
            MyTable().filter(id__ig=[1])

    def test_in(self):
        sql = "SELECT * FROM MyTable WHERE id IN (1, 2);"
        q = MyTable().filter(id__in=[1, 2]).sql
        self.assertEqual(q, sql)

        with self.assertRaises(ValueError):
            MyTable().filter(id__in="wrong")

    def test_not_in(self):
        sql = "SELECT * FROM MyTable WHERE id NOT IN (1, 2);"
        q = MyTable().filter(id__notin=[1, 2]).sql
        self.assertEqual(q, sql)

        with self.assertRaises(ValueError):
            MyTable().filter(id__notin="wrong")

    def test_eq(self):
        q1 = MyTable().filter(id__eq=1).sql
        q2 = MyTable().filter(date__eq="2020-28-11").sql
        q3 = MyTable().filter(url__eq="https://test.com").sql
        q4 = MyTable().filter(rating__eq=3.5).sql
        self.assertEqual(q1, "SELECT * FROM MyTable WHERE id = 1;")
        self.assertEqual(q2, "SELECT * FROM MyTable WHERE date = '2020-28-11';")
        self.assertEqual(q3, "SELECT * FROM MyTable WHERE url = 'https://test.com';")
        self.assertEqual(q4, "SELECT * FROM MyTable WHERE rating = 3.5;")

    def test_lt(self):
        q1 = MyTable().filter(date__lt="2020-28-11").sql
        q2 = MyTable().filter(rating__lt=1.4).sql
        self.assertEqual(q1, "SELECT * FROM MyTable WHERE date < '2020-28-11';")
        self.assertEqual(q2, "SELECT * FROM MyTable WHERE rating < 1.4;")

    def test_gt(self):
        q1 = MyTable().filter(date__gt="2020-28-11").sql
        q2 = MyTable().filter(id__gt=3).sql
        self.assertEqual(q1, "SELECT * FROM MyTable WHERE date > '2020-28-11';")
        self.assertEqual(q2, "SELECT * FROM MyTable WHERE id > 3;")

    def test_chained_query(self):
        q = (
            MyTable()
            .filter(id__notin=[1, 7], url__eq="https://test.com", date__lt="2020-28-11")
            .filter(rating__gt=2)
            .sql
        )
        self.assertEqual(
            q,
            "SELECT * FROM MyTable WHERE id NOT IN (1, 7) AND url = 'https://test.com' AND date < '2020-28-11' AND rating > 2;",
        )


suite = unittest.TestLoader().loadTestsFromTestCase(TestQuery)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)


test_chained_query (__main__.TestQuery) ... ok
test_eq (__main__.TestQuery) ... ok
test_gt (__main__.TestQuery) ... ok
test_in (__main__.TestQuery) ... ok
test_lt (__main__.TestQuery) ... ok
test_not_in (__main__.TestQuery) ... ok
test_wrong_field (__main__.TestQuery) ... ok
test_wrong_method (__main__.TestQuery) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.016s

OK


<unittest.runner.TextTestResult run=8 errors=0 failures=0>